# Configuração e teste
Aqui o script instala o ferramental necessário para o experimento.


1.   Acesso a api-key do grok (foi a llm utilizada no vídeo)
2.   Instalação da dependência da api do groq. (Ela utilizará a api-key pra se comunicar com os servidores do grok)

Depois é realizado uma interação com a api do grok

In [39]:
!pip install groq

In [40]:
import os

from google.colab import userdata
from groq import Groq

llm_api_key = userdata.get('grok_api_key')

client = Groq(
    api_key = llm_api_key,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Language models are computationally intensive, requiring significant resources to process and generate text. Fast language models enable efficient processing, reducing the time and computational resources required to complete tasks. This efficiency is critical in applications where real-time responses are essential, such as virtual assistants, chatbots, and language translation systems.
2. **Real-time Interactions**: Fast language models facilitate real-time interactions between humans and machines. They enable instant responses to user queries, making conversations more natural and engaging. This is particularly important in applications like customer service, where quick and accurate responses can significantly improve user experience.
3. **Scalability**: Fast language models can handle a large volume of requests, making the

# Implementação do agente e do System Prompt

Aqui mantivemos o mesmo agente do experimento original.

O agente é definido com a estrutura de construção, inicializando seu estado, e uma function call para registrar as entradas na pilha de interação/conversa. Adicionalmente, há um método execute que dispara a chamada com pilha de conversas para a api de chat completion da llm, no caso o grok.

In [41]:
import re

class Agent:

  def __init__(self, client, system):
    self.client = client
    self.system = system
    self.messages = []
    if self.system is not None:
      self.messages.append({"role": "system", "content": self.system})


  def __call__(self, message = ""):
    if message is not None:
      self.messages.append({"role": "user", "content": message})

    result = self.execute()

    self.messages.append({"role": "assistant", "content": result})
    return result


  def execute(self):
    completion = self.client.chat.completions.create(
      messages=self.messages,
      model="llama-3.3-70b-versatile",
    )
    return completion.choices[0].message.content


  def loop(self, max_iterations=10, query: str = ""):

    available_tools = {
        "get_menu": get_menu,
        "calculate": calculate
    }

    i = 0

    next_prompt = query

    while i < max_iterations:
        i += 1
        result = self.__call__(next_prompt)
        print("")
        print(f"--- Iteração {i} ---")
        print(result)

        if "PAUSE" in result and "Action" in result:
            # REGRA REGEX:
            # 1. ([a-z_]+) -> Nome da ferramenta
            # 2. (?::\s*(.+))? -> Opcional. Procura ':' seguido de espaço e o argumento
            match = re.search(r"Action: ([a-z_]+)(?::\s*(.+))?", result, re.IGNORECASE)

            if match:
                chosen_tool = match.group(1)
                arg = match.group(2)

                if chosen_tool in available_tools:

                    if chosen_tool == "get_menu":
                      result_tool = available_tools[chosen_tool]()

                      # Ou chamando por:
                      # result_tool = get_menu()

                    else:
                      result_tool = available_tools[chosen_tool](arg)

                      # Ou chamando por:
                      # result_tool = calculate(arg)

                    next_prompt = f"Observation: {result_tool}"
                else:
                    next_prompt = "Observation: Tool not found"

                print(next_prompt)
                continue
            else:
                print("Error: Could not parse Action from agent response.")
                break

        if "Answer" in result:
            break


## Definição do system prompt
O system prompt foi modificado para atender a regra do experimento atual. Basicamente é um atendente de uma pizzaria que utiliza duas tools (get_menu e calculate).

In [42]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your goal is to act as a cashier for a Pizza place.
Rules:
1. You must always retrieve the current menu prices before calculating.
2. If the customer eats at the restaurant (dine-in), you MUST add a 10% service fee to the total.
3. If the order is for takeout/delivery, there is NO service fee.
4. Final Answer must state the items ordered and the final total price.

Your available actions are:

get_menu:
e.g. get_menu
Returns a JSON object containing the available pizzas and sodas with their respective prices.

calculate:
e.g. calculate: 12 + 3
Runs a calculation and returns the number. Use Python syntax.

Example session 1:

Question: I want one Pepperoni pizza and a Coke to go.
Thought: I need to check the prices for Pepperoni and Coke.
Action: get_menu
PAUSE

You will be called again with this:

Observation: {"pizzas": {"Pepperoni": 15.00, "Cheese": 12.00}, "sodas": {"Coke": 3.00, "Sprite": 3.00}}

Thought: Pepperoni is 15.00. Coke is 3.00. The customer said "to go", so there is no service fee. I need to sum the prices.
Action: calculate: 15 + 3
PAUSE

You will be called again with this:

Observation: 18.0

Thought: The calculation is complete. I have the final total.
Answer: You ordered a Pepperoni pizza and a Coke. The total is $18.00.

Example session 2:

Question: I'll have a Cheese pizza and a Sprite. I'm eating here.
Thought: I need to check prices.
Action: get_menu
PAUSE

You will be called again with this:

Observation: {"pizzas": {"Pepperoni": 15.00, "Cheese": 12.00}, "sodas": {"Coke": 3.00, "Sprite": 3.00}}

Thought: Cheese is 12.00. Sprite is 3.00. The customer is "eating here", so I must add 10% to the total. Calculation is (12 + 3) * 1.10.
Action: calculate: (12 + 3) * 1.10
PAUSE

You will be called again with this:

Observation: 16.5

Thought: The total includes the service fee.
Answer: You ordered a Cheese pizza and a Sprite for dine-in. The total is $16.50.

Now it's your turn:

""".strip()

## Implementação das tools
Segue implementação das tools definidas no system prompt. Aqui surgiu uma dificuldade de implementação porque suas assinaturas são diferentes, então precisam ser tratadas de maneira diferente.

In [43]:
# Tools


import json

def get_menu():
    menu_data = {
        "pizzas": {
            "Pepperoni": 15.00,
            "Cheese": 12.00
        },
        "sodas": {
            "Coke": 3.00,
            "Sprite": 3.00
        }
    }
    return json.dumps(menu_data)


def calculate(expression):
    try:
        expression = expression.strip()
        allowed_names = {"__builtins__": None}
        result = eval(expression, allowed_names)

        return str(result)

    except Exception as e:
        return f"Error: {str(e)}"

# Definição e execução do loop

In [44]:
import re

agente = Agent(client=client, system=system_prompt)
agente.loop(query="I want a Cheese pizza and a Coke. I will be eating at the restaurant.")


--- Iteração 1 ---
Thought: I need to check the prices for Cheese pizza and Coke. Since the customer is eating at the restaurant, I will also need to add a 10% service fee to the total.

Action: get_menu
PAUSE
Observation: {"pizzas": {"Pepperoni": 15.0, "Cheese": 12.0}, "sodas": {"Coke": 3.0, "Sprite": 3.0}}

--- Iteração 2 ---
Thought: The price of a Cheese pizza is 12.0 and the price of a Coke is 3.0. Since the customer is eating at the restaurant, I need to calculate the total with a 10% service fee. The calculation is (12 + 3) * 1.10.

Action: calculate: (12 + 3) * 1.10
PAUSE
Observation: 16.5

--- Iteração 3 ---
Thought: The calculation is complete. I have the final total, which includes the 10% service fee for dining in. The total is $16.50.

Answer: You ordered a Cheese pizza and a Coke for dine-in. The total is $16.50.
